<a href="https://colab.research.google.com/github/liwenjing2012/Cousera_Capstone/blob/main/Capstone_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cousera Capstone Project : Neighborhoods Battle

### *Which neighborhood has more potentials for housing investment in the city of Austin:*

The project compares neighborhoods in the city of Austin in the following aspects


1.   Safety : criminal cases
2.   Information including household incomes and attitudes towards the neighborhood of the residents
1.   Nearby venues






In [ ]:
#install modules needed
!pip install rtree
!pip install geopandas

In [23]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
from shapely.geometry import Point
import geopandas as gpd
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # plotting library
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import seaborn as sns

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Import criminal report data from github, the data contained criminal cases occured in the city of Austin during the previous year (6/1/2020-6/1/2021)

In [100]:
url = "https://raw.githubusercontent.com/liwenjing2012/Cousera_Capstone/main/Crime_Reports.csv"
criminal_data = pd.read_csv(url)
#criminal_data.head()
criminal_features = ['Incident Number','Highest Offense Description', 'Category Description', 'Location Type', 'Zip Code', 'Latitude', 'Longitude']
criminal_feature_data = criminal_data[criminal_features]

criminal_feature_data['Category']=criminal_feature_data['Category Description'].fillna('Unlabled')
criminal_feature_data = criminal_feature_data.drop(['Category Description','Highest Offense Description'],axis=1)
criminal_feature_data.groupby(['Category']).count()

url = 'https://raw.githubusercontent.com/liwenjing2012/Cousera_Capstone/main/HousingSales.csv'
house = pd.read_csv(url)
house = house.rename(columns = {'ZIP':'Zip Code'})
house['Zip Code'] = house['Zip Code'].astype(float)

criminal_feature_data1 = pd.merge(criminal_feature_data,house,how = 'left', on = ['Zip Code'])
criminal_feature_data1 = criminal_feature_data1.fillna(0)



criminal_gdf = gpd.GeoDataFrame(
    criminal_feature_data1, geometry=gpd.points_from_xy(criminal_feature_data.Longitude, criminal_feature_data.Latitude))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
criminal_gdf.shape

Import neighborhood data from github and merge the two datasets

In [114]:
url = 'https://raw.githubusercontent.com/liwenjing2012/Cousera_Capstone/main/Neighborhoods.geojson'
geoNeigh = gpd.read_file(url)

geoNeigh_data = geoNeigh[['neighname','geometry']]

geoNeigh_data['Lng'] = geoNeigh_data['geometry'].centroid.x
geoNeigh_data['Lat'] = geoNeigh_data['geometry'].centroid.y
geoNeigh_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  import sys


,neighname,geometry,Lng,Lat
0,ANDERSON MILL,"MULTIPOLYGON (((-97.79231 30.45671, -97.79154 ...",-97.809771,30.441196
1,WINDSOR PARK,"MULTIPOLYGON (((-97.67076 30.30854, -97.67204 ...",-97.692326,30.311575
2,DAWSON,"MULTIPOLYGON (((-97.75353 30.23876, -97.75472 ...",-97.761797,30.232695
3,WEST UNIVERSITY,"MULTIPOLYGON (((-97.73815 30.30275, -97.73860 ...",-97.745308,30.291838
4,MLK,"MULTIPOLYGON (((-97.68262 30.28587, -97.68372 ...",-97.692304,30.283831


In [115]:
criminal_gdf.set_crs(epsg=4326, inplace=True)
joined_gdf = gpd.sjoin(criminal_gdf, geoNeigh_data, op='within').reset_index(drop=True).drop(['index_right'],axis=1)
joined_gdf = joined_gdf.drop(['Location Type','Zip Code','Latitude','Longitude','geometry'],axis = 1)

In [125]:
jd_crime = joined_gdf.groupby(['neighname'])['Incident Number'].count().reset_index()
jd_crime = jd_crime.rename(columns = {'Incident Nubmer':'Total Crime'})
jd = joined_gdf.drop(['Incident Number','Category','Lng','Lat'],axis = 1).groupby(['neighname']).mean().reset_index()
jd1 = pd.merge(jd_crime,jd,how = 'left', on = ['neighname'])
jd1

,neighname,Incident Number,Median_Sales_Price_2015,Total_Change_in_Median_Sales_Price,Pct_change_in_Median_Sales_Price,Total_Pop2015,PctPopChange
0,ALLANDALE,698,356966.461318,84953.120344,0.339026,25323.404011,0.020258
1,ANDERSON MILL,1402,320234.883024,73222.933666,0.299451,24822.297432,0.065713
2,AVERY RANCH--LAKELINE,1414,218997.825318,54888.914427,0.227546,19731.683168,0.151697
3,BARTON CREEK MALL,428,844073.247664,260440.070093,0.448949,27105.521028,0.019953
4,BARTON HILLS,654,601177.752294,193667.813456,0.485657,37941.795107,0.083012
...,...,...,...,...,...,...,...
97,WINDSOR HILLS,957,0.000000,0.000000,0.000000,0.000000,0.000000
98,WINDSOR PARK,2600,329365.384615,141726.923077,0.758538,34289.930769,0.219577
99,WINDSOR ROAD,226,493659.292035,145683.628319,0.363673,26981.738938,0.090575
100,WOOTEN,688,350889.244186,82946.263081,0.339506,25382.053779,0.019971


In [8]:
df = joined_gdf.copy()
df1 = df[['neighname','Lat','Lng']].drop_duplicates().reset_index(drop = True)
for name in criminal_feature_data['Category'].unique():
  if  name == 'Rape': #Rape has no lat & long
    name = 'Total'
    df[name] = df.groupby(['neighname'])['Incident Number'].transform('count')
    
  else:
    df[name] = df[df['Category']==name].groupby(['neighname'])['Incident Number'].transform('count')
    
  df_temp = df[['neighname',name]].drop_duplicates().reset_index(drop = True)
  df_temp = df_temp[-df_temp[name].isna()].reset_index(drop = True)
  df1 = pd.merge(df1,df_temp,how = 'left', on = ['neighname'])
                 

  df1 = df1.fillna(0)
  df1 = df1.astype({name:'int32'})



df1.head()

,neighname,Lat,Lng,Robbery,Unlabled,Burglary,Theft,Aggravated Assault,Auto Theft,Total,Murder
0,WINDSOR HILLS,30.363944,-97.676293,15,588,54,212,38,49,957,1
1,CHERRY CREEK,30.193485,-97.825970,2,979,41,333,30,32,1418,1
2,MLK,30.283831,-97.692304,7,550,38,114,41,17,767,0
3,DAWSON,30.232695,-97.761797,7,310,23,154,8,22,524,0
4,WEST UNIVERSITY,30.291838,-97.745308,15,673,83,706,15,149,1641,0


In [ ]:
map_austin = folium.Map(location=[30.2672, -97.7431], zoom_start=11)

# add markers to map
for lat, lng, neighborhood,cnt in zip(df1['Lat'], df1['Lng'], df1['neighname'],df1['Total']):
    label = '{},{} crimes'.format(neighborhood,cnt)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_austin)  
    
map_austin

Import survey data of the city of Austin from 2015-2019

In [ ]:
url = 'https://raw.githubusercontent.com/liwenjing2012/Cousera_Capstone/main/Community_Survey.csv'
survey_data = pd.read_csv(url)
survey_data.head()

In [ ]:
url = 'https://raw.githubusercontent.com/liwenjing2012/Cousera_Capstone/main/Community_Survey.csv'
survey_data = pd.read_csv(url)

features = ['ID','The City of Austin as a place to live',        
            'The City of Austin as a place to work','The City of Austin as a place to raise children',
            'The City of Austin as a place to retire','Overall quality of life in the city',
            'Block Level X Longitude','Block Level Y Latitude','Do you own or rent your home?',
            'Which of the following best describes your annual household income?']
survey_data = survey_data[features]
survey_data = survey_data.rename(columns={'The City of Austin as a place to live': "Live_Satisfy", 
                            "The City of Austin as a place to work": "work_Satisfy",
                            'The City of Austin as a place to raise children':"children_raise",
                            'The City of Austin as a place to retire': "retire_satisfy",
                            'Overall quality of life in the city':"overall_quality",
                            'Do you own or rent your home?':'own_rent',
                            'Which of the following best describes your annual household income?':'income_level',
                            'Block Level X Longitude':'Longitude',
                            'Block Level Y Latitude':'Latitude'
                            })

survey_gdf = gpd.GeoDataFrame(
    survey_data, geometry=gpd.points_from_xy(survey_data.Longitude, survey_data.Latitude))
survey_gdf.set_crs(epsg=4326, inplace=True)
joined_survey = gpd.sjoin(survey_gdf, geoNeigh_data, op='within').reset_index(drop=True).drop(['index_right'],axis=1)
joined_survey = joined_survey.drop(['Latitude','Longitude','geometry'],axis = 1)
joined_survey.head()


In [128]:
joined_survey.groupby('Live_Satisfy').count()
#Don't Know --> 1
#Dissatisfied --> 1
#Neutral --> 2
#Satisfied --> 3
#Very Dissatisfied --> 0
#Very Satisfied --> 4

joined_survey = joined_survey.replace({"Don't Know":1,'Dissatisfied':1,
                                       "Very Dissatisfied":0,"Neutral":2,
                                       "Satisfied":3,"Very Satisfied":4,
                                       'Not provided':'Not Provided'})

joined_survey.groupby('income_level').count()
#Not Provided -->0
#less than $20K --> 1
#$20K-39,999-->2
#$40K-59,999-->3
#$60K-79,999-->4
#$80K-149,999-->5
#$150K or more-->6
joined_survey = joined_survey.replace({"Not Provided":0,"less than $20K":1,
                                       "$20K-39,999":2,"$40K-59,999":3,
                                       "$60K-79,999":4,"$80K-149,999":5,
                                       "$150K or more":6})
joined_survey = joined_survey.groupby('neighname').mean().reset_index()
joined_survey = joined_survey.drop(['Lng','Lat'],axis = 1)
#joined_survey
full_joined = pd.merge(jd1,joined_survey,how = 'left', on = ['neighname'])
full_joined

,neighname,Incident Number,Median_Sales_Price_2015,Total_Change_in_Median_Sales_Price,Pct_change_in_Median_Sales_Price,Total_Pop2015,PctPopChange,Live_Satisfy,work_Satisfy,children_raise,retire_satisfy,overall_quality,income_level
0,ALLANDALE,698,356966.461318,84953.120344,0.339026,25323.404011,0.020258,2.991597,2.932773,2.436975,1.815126,2.815126,4.033613
1,ANDERSON MILL,1402,320234.883024,73222.933666,0.299451,24822.297432,0.065713,3.003992,2.930140,2.768463,2.071856,2.812375,3.854291
2,AVERY RANCH--LAKELINE,1414,218997.825318,54888.914427,0.227546,19731.683168,0.151697,2.921053,2.831579,2.684211,1.984211,2.663158,4.068421
3,BARTON CREEK MALL,428,844073.247664,260440.070093,0.448949,27105.521028,0.019953,3.233010,3.106796,2.854369,2.349515,3.048544,4.252427
4,BARTON HILLS,654,601177.752294,193667.813456,0.485657,37941.795107,0.083012,3.266055,3.110092,2.587156,2.137615,3.045872,4.055046
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,WINDSOR HILLS,957,0.000000,0.000000,0.000000,0.000000,0.000000,2.923810,2.838095,2.542857,2.123810,2.609524,2.857143
98,WINDSOR PARK,2600,329365.384615,141726.923077,0.758538,34289.930769,0.219577,2.899563,2.764192,2.340611,1.855895,2.694323,3.393013
99,WINDSOR ROAD,226,493659.292035,145683.628319,0.363673,26981.738938,0.090575,3.062500,2.875000,3.046875,2.078125,2.859375,4.687500
100,WOOTEN,688,350889.244186,82946.263081,0.339506,25382.053779,0.019971,2.796296,2.796296,2.296296,1.685185,2.740741,3.333333


Import high school graduating data 

In [129]:
kclusters = 3

df2 = full_joined.copy()
df_clustering = df2.drop(['neighname'],1)
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(df_clustering)
kmeans.labels_
df2.insert(0,'crime_clusters',kmeans.labels_)

df2





,crime_clusters,neighname,Incident Number,Median_Sales_Price_2015,Total_Change_in_Median_Sales_Price,Pct_change_in_Median_Sales_Price,Total_Pop2015,PctPopChange,Live_Satisfy,work_Satisfy,children_raise,retire_satisfy,overall_quality,income_level
0,1,ALLANDALE,698,356966.461318,84953.120344,0.339026,25323.404011,0.020258,2.991597,2.932773,2.436975,1.815126,2.815126,4.033613
1,1,ANDERSON MILL,1402,320234.883024,73222.933666,0.299451,24822.297432,0.065713,3.003992,2.930140,2.768463,2.071856,2.812375,3.854291
2,1,AVERY RANCH--LAKELINE,1414,218997.825318,54888.914427,0.227546,19731.683168,0.151697,2.921053,2.831579,2.684211,1.984211,2.663158,4.068421
3,0,BARTON CREEK MALL,428,844073.247664,260440.070093,0.448949,27105.521028,0.019953,3.233010,3.106796,2.854369,2.349515,3.048544,4.252427
4,0,BARTON HILLS,654,601177.752294,193667.813456,0.485657,37941.795107,0.083012,3.266055,3.110092,2.587156,2.137615,3.045872,4.055046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2,WINDSOR HILLS,957,0.000000,0.000000,0.000000,0.000000,0.000000,2.923810,2.838095,2.542857,2.123810,2.609524,2.857143
98,1,WINDSOR PARK,2600,329365.384615,141726.923077,0.758538,34289.930769,0.219577,2.899563,2.764192,2.340611,1.855895,2.694323,3.393013
99,0,WINDSOR ROAD,226,493659.292035,145683.628319,0.363673,26981.738938,0.090575,3.062500,2.875000,3.046875,2.078125,2.859375,4.687500
100,1,WOOTEN,688,350889.244186,82946.263081,0.339506,25382.053779,0.019971,2.796296,2.796296,2.296296,1.685185,2.740741,3.333333


In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
df3 = df2[['crime_clusters','Robbery']]
df3
#ax.boxplot(df2[['clusters','Robbery']])
#plt.show()
sns.boxplot(x="crime_clusters", y="Robbery", data=df2)

In [ ]:
sns.boxplot(x="clusters", y="Murder", data=df2)

In [ ]:
sns.boxplot(x="clusters", y="Auto Theft", data=df2)

In [ ]:
map_clusters = folium.Map(location=[30.2672, -97.7431],zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
#ys
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lng, neighborhood,cluster in zip(df1['Lat'],df1['Lng'],df2['neighname'],df2['crime_clusters']):
  label = folium.Popup(str(neighborhood) + 'Cluster' + str(cluster), parse_html=True)
  folium.CircleMarker(
      [lat,lng],
      radius = 5,
      popup = label,
      color = rainbow[cluster-1],
      fill = True,
      fill_color = rainbow[cluster-1],
      fill_opacity = 0.7).add_to(map_clusters)

map_clusters

In [ ]:
CLIENT_ID = "KU0CC2XGBFTS40SCQZYT5C1W4JMNIRA1D5H3SVZCITNDO3XF"
CLIENT_SECRET="HCOL1SIL4HGTJKTXPG34GVCF0YIPNY4PIOTVLDTERRFQRZYD"
VERSION = '20180604'
ACCESS_TOKEN = '0GFYVQWD4RZ2YPEWEFDYCKM0FN5ADDL1KVYPC1PWASSNHZBH'

LIMIT = 100

In [ ]:
# Create function that gets the top 100 venues that are in the neighbourhoods within a radius of 500 meters 

radius = 1650
venue_list=[]
for name, lat, lng in zip(geoNeigh_data['neighname'], geoNeigh_data['Lat'], geoNeigh_data['Lng']):
    print(name)
        
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius,
        LIMIT)
        
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    venue_list.append([(
        name, 
        lat, 
        lng, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venue_list for item in venue_list])
nearby_venues.columns = ['neighname', 
              'neighlat', 
              'neighlng', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']


In [ ]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 347 uniques categories.


In [ ]:
hotvenues = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hotvenues['neighname'] = nearby_venues['neighname'] 

# move neighborhood column to the first column
fixed_columns = [hotvenues.columns[-1]] + list(hotvenues.columns[:-1])
austin_onehot = hotvenues[fixed_columns]
austin_onehot


austin_grouped = austin_onehot.groupby('neighname').sum().reset_index()
austin_grouped

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
    
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighname']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighname'] = austin_grouped['neighname']

for ind in np.arange(austin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(austin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,10) 
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(chi_cluster) 
    kmeanModel.fit(chi_cluster)     
      
    distortions.append(sum(np.min(cdist(chi_cluster, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / chi_cluster.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(chi_cluster, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / chi_cluster.shape[0] 
    mapping2[k] = kmeanModel.inertia_ 

for key,val in mapping1.items(): 
    print(str(key)+' : '+str(val)) 

plt.plot(K, distortions, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method Using Distortion') 
plt.show() 